In [2]:
import pandas as pd
import random
from datetime import datetime, timedelta

# --- Main Generation Function ---
def generate_mandal_dataset(mandal_name, city, base_headcount_range, yearly_growth_factor, start_year=2010, num_years=15):
    """
    Generates a realistic dummy dataset with binary flags for weekend and special days.
    """
    print(f"--- Generating dataset for: {mandal_name}, {city} ---")

    FESTIVAL_DURATION_DAYS = 11
    weather_options = ['Humid', 'Cloudy', 'Sunny', 'Light Rain', 'Heavy Rain']
    all_data = []
    total_hours = FESTIVAL_DURATION_DAYS * 24

    for year_offset in range(num_years):
        current_year = start_year + year_offset
        festival_start_date = datetime(current_year, 9, 5, 0, 0, 0)

        for hour_offset in range(total_hours):
            current_time = festival_start_date + timedelta(hours=hour_offset)

            day_of_festival = (current_time - festival_start_date).days + 1
            hour_of_day = current_time.hour

            # --- MODIFICATION START ---

            # 1. Determine day of week for headcount logic, then create binary flag
            day_of_week_str = current_time.strftime('%A')
            is_weekend = 1 if day_of_week_str in ['Saturday', 'Sunday'] else 0

            # 2. Determine special day type for headcount logic, then create binary flag
            if day_of_festival == 1: special_day_str = 'Pratishthapana'
            elif day_of_festival == 11: special_day_str = 'Visarjan'
            elif day_of_festival in [5, 7, 10]: special_day_str = 'Key Day'
            else: special_day_str = 'Regular Day'
            is_special_day = 1 if special_day_str != 'Regular Day' else 0

            # --- MODIFICATION END ---

            weather = random.choice(weather_options)

            # Headcount Simulation (uses the original string variables for logic)
            base_headcount = random.randint(*base_headcount_range)
            base_headcount += (current_year - start_year) * yearly_growth_factor
            headcount = float(base_headcount)

            if 18 <= hour_of_day <= 23: headcount *= random.uniform(2.5, 4.0)
            elif 1 <= hour_of_day <= 5: headcount *= random.uniform(0.1, 0.4)

            if special_day_str == 'Visarjan': headcount *= random.uniform(3.5, 5.0)
            elif special_day_str == 'Pratishthapana': headcount *= random.uniform(2.0, 3.0)

            if is_weekend == 1: # Logic now uses the binary flag
                headcount *= random.uniform(1.4, 2.0)

            if weather == 'Heavy Rain': headcount *= 0.4
            elif weather == 'Light Rain': headcount *= 0.7

            # Append the new binary columns to the data
            all_data.append([
                mandal_name, city, current_time, current_year, day_of_festival,
                hour_of_day, is_weekend, is_special_day, weather, int(headcount)
            ])

    # Update the column names for the final DataFrame
    columns = [
        'mandal_name', 'city', 'datetime', 'year', 'day_of_festival', 'hour_of_day',
        'is_weekend', 'is_special_day', 'weather', 'headcount'
    ]
    df = pd.DataFrame(all_data, columns=columns)

    filename = f"{mandal_name.lower().replace(' ', '_')}_binary_dataset.csv"
    df.to_csv(filename, index=False)
    print(f"Successfully created '{filename}' with {len(df)} rows.")
    # Show a preview of the first file generated
    if mandal_name == mandals_to_generate[0]["mandal_name"]:
        print("\nPreview of the new format:")
        print(df.head())
        print("\n")


# --- Configuration for 5 Mandals ---
mandals_to_generate = [
    {
        "mandal_name": "Lalbaugcha Raja",
        "city": "Mumbai",
        "base_headcount_range": (10000, 25000),
        "yearly_growth_factor": 1800
    },
    {
        "mandal_name": "Dagdusheth Halwai Ganpati",
        "city": "Pune",
        "base_headcount_range": (8000, 20000),
        "yearly_growth_factor": 1500
    },
    {
        "mandal_name": "Andhericha Raja",
        "city": "Mumbai",
        "base_headcount_range": (4000, 9000),
        "yearly_growth_factor": 1000
    },
    {
        "mandal_name": "Kasba Ganpati",
        "city": "Pune",
        "base_headcount_range": (3000, 7000),
        "yearly_growth_factor": 700
    },
    {
        "mandal_name": "Siddhivinayak Temple",
        "city": "Mumbai",
        "base_headcount_range": (7000, 15000),
        "yearly_growth_factor": 1200
    }
]

# --- Main Execution Block ---
if __name__ == "__main__":
    print("Starting dataset generation with binary columns...")
    for mandal_config in mandals_to_generate:
        generate_mandal_dataset(**mandal_config)
    print("All datasets have been generated successfully.")

Starting dataset generation with binary columns...
--- Generating dataset for: Lalbaugcha Raja, Mumbai ---
Successfully created 'lalbaugcha_raja_binary_dataset.csv' with 3960 rows.

Preview of the new format:
       mandal_name    city            datetime  year  day_of_festival  \
0  Lalbaugcha Raja  Mumbai 2010-09-05 00:00:00  2010                1   
1  Lalbaugcha Raja  Mumbai 2010-09-05 01:00:00  2010                1   
2  Lalbaugcha Raja  Mumbai 2010-09-05 02:00:00  2010                1   
3  Lalbaugcha Raja  Mumbai 2010-09-05 03:00:00  2010                1   
4  Lalbaugcha Raja  Mumbai 2010-09-05 04:00:00  2010                1   

   hour_of_day  is_weekend  is_special_day     weather  headcount  
0            0           1               1       Humid      79170  
1            1           1               1  Heavy Rain       4376  
2            2           1               1       Sunny      13827  
3            3           1               1      Cloudy      12522  
4           

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import pandas as pd
import glob

# --- Step 1: Combine All Datasets ---

# Find all CSV files that end with '_binary_dataset.csv'
file_pattern = '*_binary_dataset.csv'
all_files = glob.glob(file_pattern)

print(f"Found {len(all_files)} files to combine:")
print(all_files)

# Load each file into a DataFrame and store them in a list
list_of_dfs = [pd.read_csv(f) for f in all_files]

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(list_of_dfs, ignore_index=True)

print(f"\nShape of the combined dataset before sorting: {combined_df.shape}")

# --- Step 2: Prepare and Sort the Combined Data ---

# Convert 'datetime' column to a proper datetime object
combined_df['datetime'] = pd.to_datetime(combined_df['datetime'])

# CRITICAL: Sort the entire DataFrame by datetime to ensure chronological order
combined_df = combined_df.sort_values(by='datetime').reset_index(drop=True)

print("\nCombined dataset has been sorted chronologically.")
print("Preview of the first few rows (start of 2010):")
print(combined_df.head())
print("\nPreview of the last few rows (end of 2024):")
print(combined_df.tail())


# --- Step 3: Perform the Chronological Split ---

# Define split points (70% train, 15% validation, 15% test)
train_split_index = int(len(combined_df) * 0.70)
validation_split_index = int(len(combined_df) * 0.85)

# Create the splits
train_data = combined_df.iloc[:train_split_index]
validation_data = combined_df.iloc[train_split_index:validation_split_index]
test_data = combined_df.iloc[validation_split_index:]

print("\n--- Data Split Complete ---")
print(f"Training data shape:   {train_data.shape}")
print(f"Validation data shape: {validation_data.shape}")
print(f"Test data shape:       {test_data.shape}")
print("---")
print(f"Training data dates:   {train_data['datetime'].min()} to {train_data['datetime'].max()}")
print(f"Validation data dates: {validation_data['datetime'].min()} to {validation_data['datetime'].max()}")
print(f"Test data dates:       {test_data['datetime'].min()} to {test_data['datetime'].max()}")

Found 5 files to combine:
['andhericha_raja_binary_dataset.csv', 'siddhivinayak_temple_binary_dataset.csv', 'dagdusheth_halwai_ganpati_binary_dataset.csv', 'lalbaugcha_raja_binary_dataset.csv', 'kasba_ganpati_binary_dataset.csv']

Shape of the combined dataset before sorting: (19800, 10)

Combined dataset has been sorted chronologically.
Preview of the first few rows (start of 2010):
                 mandal_name    city   datetime  year  day_of_festival  \
0            Andhericha Raja  Mumbai 2010-09-05  2010                1   
1  Dagdusheth Halwai Ganpati    Pune 2010-09-05  2010                1   
2            Lalbaugcha Raja  Mumbai 2010-09-05  2010                1   
3       Siddhivinayak Temple  Mumbai 2010-09-05  2010                1   
4              Kasba Ganpati    Pune 2010-09-05  2010                1   

   hour_of_day  is_weekend  is_special_day     weather  headcount  
0            0           1               1       Sunny      24036  
1            0           1      

In [8]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

# Assume 'train_data', 'validation_data', and 'test_data' are already created
# from the previous step.

# --- Step 1: Feature Engineering (Adding Lag Features) ---
def create_features(df):
    """Create time series features based on datetime index."""
    df = df.copy()
    # Create a 1-hour lag and a 24-hour lag
    df['headcount_lag_1hr'] = df.groupby('mandal_name')['headcount'].shift(1)
    df['headcount_lag_24hr'] = df.groupby('mandal_name')['headcount'].shift(24)
    return df

train_featured = create_features(train_data)
validation_featured = create_features(validation_data)
test_featured = create_features(test_data)


# --- Step 2: Preprocessing ---

# Define which columns are features and which is the target
TARGET = 'headcount'
# Drop the original datetime and any rows with missing lag values
FEATURES = ['year', 'day_of_festival', 'hour_of_day', 'is_weekend',
            'is_special_day', 'mandal_name', 'city', 'weather',
            'headcount_lag_1hr', 'headcount_lag_24hr']

# Create X (features) and y (target) sets
X_train = train_featured[FEATURES].dropna()
y_train = train_featured.loc[X_train.index][TARGET]

X_val = validation_featured[FEATURES].dropna()
y_val = validation_featured.loc[X_val.index][TARGET]

X_test = test_featured[FEATURES].dropna()
y_test = test_featured.loc[X_test.index][TARGET]


# One-Hot Encode categorical features
X_train = pd.get_dummies(X_train, drop_first=True)
X_val = pd.get_dummies(X_val, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Align columns - ensures val/test sets have the same columns as the train set
# after one-hot encoding, filling missing ones with 0.
train_cols = X_train.columns
X_val = X_val.reindex(columns=train_cols, fill_value=0)
X_test = X_test.reindex(columns=train_cols, fill_value=0)


# --- Step 3 & 4: Model Training and Evaluation ---

# Initialize and train the LightGBM Regressor model
model = lgb.LGBMRegressor(objective='mae',
                          metric='mae',
                          n_estimators=1000,
                          n_jobs=-1,
                          learning_rate=0.05,
                          random_state=42)

print("Training the LightGBM model...")
model.fit(X_train, y_train,
          eval_set=[(X_val, y_val)],
          eval_metric='mae',
          callbacks=[lgb.early_stopping(100, verbose=False)])

# Make predictions on the validation data
val_predictions = model.predict(X_val)

# Evaluate the model
mae = mean_absolute_error(y_val, val_predictions)
print(f"\nModel training complete.")
print(f"The Mean Absolute Error (MAE) on the validation set is: {mae:,.2f}")
print(f"This means the model's predictions are, on average, off by about {int(round(mae, -2))} people.")

Training the LightGBM model...

Model training complete.
The Mean Absolute Error (MAE) on the validation set is: 9,309.08
This means the model's predictions are, on average, off by about 9300 people.


In [7]:
# Quick implementation script - add this to your existing notebook
# This uses your existing train_data, validation_data, test_data

# Install required packages (run this first if needed)
# !pip install optuna xgboost

import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import warnings
warnings.filterwarnings('ignore')

def create_enhanced_features(df):
    """Create enhanced features from your existing data."""
    df = df.copy()
    df = df.sort_values(['mandal_name', 'datetime']).reset_index(drop=True)

    # Basic lag features (improved)
    for lag in [1, 2, 3, 6, 12, 24, 48]:
        df[f'headcount_lag_{lag}h'] = df.groupby('mandal_name')['headcount'].shift(lag)

    # Rolling statistics
    for window in [3, 6, 12, 24]:
        df[f'headcount_rolling_mean_{window}h'] = df.groupby('mandal_name')['headcount'].transform(
            lambda x: x.rolling(window, min_periods=1).mean().shift(1)
        )
        df[f'headcount_rolling_std_{window}h'] = df.groupby('mandal_name')['headcount'].transform(
            lambda x: x.rolling(window, min_periods=1).std().shift(1)
        )

    # Cyclical time features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_festival'] / 11)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_festival'] / 11)

    # Weather impact score
    weather_impact = {
        'Sunny': 1.0, 'Humid': 0.9, 'Cloudy': 0.85,
        'Light Rain': 0.7, 'Heavy Rain': 0.4
    }
    df['weather_impact_score'] = df['weather'].map(weather_impact)

    # Interaction features
    df['hour_x_weekend'] = df['hour_of_day'] * df['is_weekend']
    df['hour_x_special'] = df['hour_of_day'] * df['is_special_day']
    df['weather_x_weekend'] = df['weather_impact_score'] * df['is_weekend']

    # Peak hours
    df['is_peak_hour'] = ((df['hour_of_day'] >= 18) & (df['hour_of_day'] <= 23)).astype(int)
    df['is_late_night'] = ((df['hour_of_day'] >= 1) & (df['hour_of_day'] <= 5)).astype(int)

    # Festival progress
    df['festival_progress'] = df['day_of_festival'] / 11
    df['days_to_visarjan'] = 11 - df['day_of_festival']

    # City encoding
    df['is_mumbai'] = (df['city'] == 'Mumbai').astype(int)

    return df

def prepare_data_for_training(train_data, validation_data, test_data):
    """Prepare data with enhanced features."""
    print("Creating enhanced features...")

    # Create features for all datasets
    train_featured = create_enhanced_features(train_data)
    val_featured = create_enhanced_features(validation_data)
    test_featured = create_enhanced_features(test_data)

    # Define feature columns
    feature_cols = [
        'year', 'day_of_festival', 'hour_of_day', 'is_weekend', 'is_special_day',
        'weather_impact_score', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
        'hour_x_weekend', 'hour_x_special', 'weather_x_weekend',
        'is_peak_hour', 'is_late_night', 'festival_progress', 'days_to_visarjan', 'is_mumbai'
    ]

    # Add lag and rolling features
    lag_cols = [col for col in train_featured.columns if 'lag_' in col or 'rolling_' in col]
    feature_cols.extend(lag_cols)

    # Encode mandal names
    le = LabelEncoder()
    train_featured['mandal_encoded'] = le.fit_transform(train_featured['mandal_name'])
    val_featured['mandal_encoded'] = le.transform(val_featured['mandal_name'])
    test_featured['mandal_encoded'] = le.transform(test_featured['mandal_name'])
    feature_cols.append('mandal_encoded')

    # Prepare datasets
    X_train = train_featured[feature_cols].fillna(method='ffill').fillna(0)
    X_val = val_featured[feature_cols].fillna(method='ffill').fillna(0)
    X_test = test_featured[feature_cols].fillna(method='ffill').fillna(0)

    y_train = train_featured['headcount']
    y_val = val_featured['headcount']
    y_test = test_featured['headcount']

    # Scale features
    scaler = RobustScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print(f"Training features shape: {X_train_scaled.shape}")
    print(f"Number of features: {len(feature_cols)}")

    return X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val, y_test, scaler

def train_enhanced_model(X_train, X_val, X_test, y_train, y_val, y_test):
    """Train the enhanced ensemble model."""

    # Individual models with better parameters
    lgb_model = lgb.LGBMRegressor(
        objective='regression', metric='mae', n_estimators=1000,
        learning_rate=0.05, max_depth=8, num_leaves=100,
        min_child_samples=20, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=1.0, reg_lambda=1.0, random_state=42, verbosity=-1
    )

    xgb_model = xgb.XGBRegressor(
        n_estimators=800, learning_rate=0.05, max_depth=8,
        min_child_weight=5, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=1.0, reg_lambda=1.0, random_state=42
    )

    rf_model = RandomForestRegressor(
        n_estimators=200, max_depth=15, min_samples_split=10,
        min_samples_leaf=5, random_state=42, n_jobs=-1
    )

    # Create ensemble
    ensemble = VotingRegressor([
        ('lgb', lgb_model),
        ('xgb', xgb_model),
        ('rf', rf_model)
    ])

    print("Training enhanced ensemble model...")
    ensemble.fit(X_train, y_train)

    # Evaluate on validation set
    val_pred = ensemble.predict(X_val)
    val_mae = mean_absolute_error(y_val, val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    val_r2 = r2_score(y_val, val_pred)

    print(f"\n=== Validation Results ===")
    print(f"MAE: {val_mae:,.2f} (Previous: 8,793)")
    print(f"RMSE: {val_rmse:,.2f}")
    print(f"R²: {val_r2:.4f}")
    print(f"Improvement: {8793 - val_mae:+.2f} MAE reduction")

    # Test set evaluation
    test_pred = ensemble.predict(X_test)
    test_mae = mean_absolute_error(y_test, test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
    test_r2 = r2_score(y_test, test_pred)

    print(f"\n=== Test Results ===")
    print(f"MAE: {test_mae:,.2f}")
    print(f"RMSE: {test_rmse:,.2f}")
    print(f"R²: {test_r2:.4f}")

    # Feature importance from LightGBM
    lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(100, verbose=False)])
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': lgb_model.feature_importances_
    }).sort_values('importance', ascending=False)

    print(f"\n=== Top 10 Important Features ===")
    print(feature_importance.head(10))

    return ensemble, feature_importance

# Execute the enhanced model
print("=== Enhanced Ganpati Festival Crowd Prediction ===")

# Prepare data
X_train, X_val, X_test, y_train, y_val, y_test, scaler = prepare_data_for_training(
    train_data, validation_data, test_data
)

# Train model
model, importance = train_enhanced_model(X_train, X_val, X_test, y_train, y_val, y_test)

print(f"\n=== Enhancement Complete ===")
print("Your model should now perform significantly better!")
print("Key improvements:")
print("- Advanced lag and rolling features")
print("- Cyclical time encoding")
print("- Weather impact scoring")
print("- Interaction features")
print("- Ensemble of 3 models")
print("- Robust scaling")

=== Enhanced Ganpati Festival Crowd Prediction ===
Creating enhanced features...
Training features shape: (13860, 34)
Number of features: 34
Training enhanced ensemble model...

=== Validation Results ===
MAE: 8,834.51 (Previous: 8,793)
RMSE: 20,412.18
R²: 0.8917
Improvement: -41.51 MAE reduction

=== Test Results ===
MAE: 11,364.67
RMSE: 31,684.74
R²: 0.8668

=== Top 10 Important Features ===
                       feature  importance
5         weather_impact_score         694
31  headcount_rolling_mean_24h         566
18            headcount_lag_1h         525
23           headcount_lag_24h         520
2                  hour_of_day         500
29  headcount_rolling_mean_12h         497
21            headcount_lag_6h         462
1              day_of_festival         444
24           headcount_lag_48h         437
22           headcount_lag_12h         433

=== Enhancement Complete ===
Your model should now perform significantly better!
Key improvements:
- Advanced lag and rolling fea

In [4]:
# Quick implementation script - add this to your existing notebook
# This uses your existing train_data, validation_data, test_data

# Install required packages (run this first if needed)
# !pip install optuna xgboost scikit-learn pandas numpy lightgbm

import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.multioutput import MultiOutputRegressor # Import MultiOutputRegressor
import warnings
warnings.filterwarnings('ignore')

def create_enhanced_features(df):
    """
    Create enhanced features from your existing data, including the new turnover target.
    """
    df = df.copy()
    df = df.sort_values(['mandal_name', 'datetime']).reset_index(drop=True)

    # --- NEW: Create the turnover target variable ---
    # This calculates the change in headcount for the next hour for each mandal
    df['turnover_next_hour'] = df.groupby('mandal_name')['headcount'].transform(
        lambda x: x.shift(-1) - x
    )

    # Basic lag features
    for lag in [1, 2, 3, 6, 12, 24, 48]:
        df[f'headcount_lag_{lag}h'] = df.groupby('mandal_name')['headcount'].shift(lag)

    # Rolling statistics
    for window in [3, 6, 12, 24]:
        df[f'headcount_rolling_mean_{window}h'] = df.groupby('mandal_name')['headcount'].transform(
            lambda x: x.rolling(window, min_periods=1).mean().shift(1)
        )
        df[f'headcount_rolling_std_{window}h'] = df.groupby('mandal_name')['headcount'].transform(
            lambda x: x.rolling(window, min_periods=1).std().shift(1)
        )

    # Cyclical time features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_festival'] / 11)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_festival'] / 11)

    # Weather impact score
    weather_impact = {
        'Sunny': 1.0, 'Humid': 0.9, 'Cloudy': 0.85,
        'Light Rain': 0.7, 'Heavy Rain': 0.4
    }
    df['weather_impact_score'] = df['weather'].map(weather_impact)

    # Interaction features
    df['hour_x_weekend'] = df['hour_of_day'] * df['is_weekend']
    df['hour_x_special'] = df['hour_of_day'] * df['is_special_day']
    df['weather_x_weekend'] = df['weather_impact_score'] * df['is_weekend']

    # Peak hours
    df['is_peak_hour'] = ((df['hour_of_day'] >= 18) & (df['hour_of_day'] <= 23)).astype(int)
    df['is_late_night'] = ((df['hour_of_day'] >= 1) & (df['hour_of_day'] <= 5)).astype(int)

    # Festival progress
    df['festival_progress'] = df['day_of_festival'] / 11
    df['days_to_visarjan'] = 11 - df['day_of_festival']

    # City encoding
    df['is_mumbai'] = (df['city'] == 'Mumbai').astype(int)

    return df

def prepare_data_for_training(train_data, validation_data, test_data):
    """Prepare data with enhanced features for multi-output training."""
    print("Creating enhanced features for headcount and turnover prediction...")

    # Create features for all datasets
    train_featured = create_enhanced_features(train_data)
    val_featured = create_enhanced_features(validation_data)
    test_featured = create_enhanced_features(test_data)

    # --- IMPORTANT: Drop rows where turnover cannot be calculated (the last entry for each mandal) ---
    train_featured = train_featured.dropna(subset=['turnover_next_hour'])
    val_featured = val_featured.dropna(subset=['turnover_next_hour'])
    test_featured = test_featured.dropna(subset=['turnover_next_hour'])

    # Define feature columns
    feature_cols = [
        'year', 'day_of_festival', 'hour_of_day', 'is_weekend', 'is_special_day',
        'weather_impact_score', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos',
        'hour_x_weekend', 'hour_x_special', 'weather_x_weekend',
        'is_peak_hour', 'is_late_night', 'festival_progress', 'days_to_visarjan', 'is_mumbai'
    ]
    lag_cols = [col for col in train_featured.columns if 'lag_' in col or 'rolling_' in col]
    feature_cols.extend(lag_cols)

    # Encode mandal names
    le = LabelEncoder()
    train_featured['mandal_encoded'] = le.fit_transform(train_featured['mandal_name'])
    val_featured['mandal_encoded'] = le.transform(val_featured['mandal_name'])
    test_featured['mandal_encoded'] = le.transform(test_featured['mandal_name'])
    feature_cols.append('mandal_encoded')

    # Define target columns
    target_cols = ['headcount', 'turnover_next_hour']

    # Prepare datasets
    X_train = train_featured[feature_cols].fillna(method='ffill').fillna(0)
    X_val = val_featured[feature_cols].fillna(method='ffill').fillna(0)
    X_test = test_featured[feature_cols].fillna(method='ffill').fillna(0)

    y_train = train_featured[target_cols]
    y_val = val_featured[target_cols]
    y_test = test_featured[target_cols]

    # Scale features
    scaler = RobustScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    print(f"Training features shape: {X_train_scaled.shape}")
    print(f"Number of features: {len(feature_cols)}")

    return X_train_scaled, X_val_scaled, X_test_scaled, y_train, y_val, y_test, scaler

def train_enhanced_model(X_train, X_val, X_test, y_train, y_val, y_test):
    """Train the enhanced multi-output ensemble model."""

    # Define individual models (these will be used for each target)
    lgb_model = lgb.LGBMRegressor(
        objective='regression', metric='mae', n_estimators=1000,
        learning_rate=0.05, max_depth=8, num_leaves=100,
        min_child_samples=20, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=1.0, reg_lambda=1.0, random_state=42, verbosity=-1
    )
    xgb_model = xgb.XGBRegressor(
        n_estimators=800, learning_rate=0.05, max_depth=8,
        min_child_weight=5, subsample=0.8, colsample_bytree=0.8,
        reg_alpha=1.0, reg_lambda=1.0, random_state=42
    )
    rf_model = RandomForestRegressor(
        n_estimators=200, max_depth=15, min_samples_split=10,
        min_samples_leaf=5, random_state=42, n_jobs=-1
    )

    # Create the base ensemble
    base_ensemble = VotingRegressor([
        ('lgb', lgb_model), ('xgb', xgb_model), ('rf', rf_model)
    ])

    # --- NEW: Wrap the ensemble in a MultiOutputRegressor ---
    multi_output_ensemble = MultiOutputRegressor(base_ensemble, n_jobs=-1)

    print("Training enhanced multi-output ensemble model...")
    multi_output_ensemble.fit(X_train, y_train)

    # --- UPDATED: Evaluate on validation set for both targets ---
    val_pred = multi_output_ensemble.predict(X_val)

    # Unpack predictions and true values
    y_val_headcount, y_val_turnover = y_val['headcount'], y_val['turnover_next_hour']
    val_pred_headcount, val_pred_turnover = val_pred[:, 0], val_pred[:, 1]

    print(f"\n=== Validation Results ===")
    print("--- Headcount Prediction ---")
    print(f"MAE: {mean_absolute_error(y_val_headcount, val_pred_headcount):,.2f} (Previous: 8,793)")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val_headcount, val_pred_headcount)):,.2f}")
    print(f"R²: {r2_score(y_val_headcount, val_pred_headcount):.4f}")

    print("\n--- Turnover Prediction ---")
    print(f"MAE: {mean_absolute_error(y_val_turnover, val_pred_turnover):,.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_val_turnover, val_pred_turnover)):,.2f}")
    print(f"R²: {r2_score(y_val_turnover, val_pred_turnover):.4f}")

    # --- UPDATED: Test set evaluation ---
    test_pred = multi_output_ensemble.predict(X_test)
    y_test_headcount, y_test_turnover = y_test['headcount'], y_test['turnover_next_hour']
    test_pred_headcount, test_pred_turnover = test_pred[:, 0], test_pred[:, 1]

    print(f"\n=== Test Results ===")
    print("--- Headcount Prediction ---")
    print(f"MAE: {mean_absolute_error(y_test_headcount, test_pred_headcount):,.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test_headcount, test_pred_headcount)):,.2f}")
    print(f"R²: {r2_score(y_test_headcount, test_pred_headcount):.4f}")

    print("\n--- Turnover Prediction ---")
    print(f"MAE: {mean_absolute_error(y_test_turnover, test_pred_turnover):,.2f}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test_turnover, test_pred_turnover)):,.2f}")
    print(f"R²: {r2_score(y_test_turnover, test_pred_turnover):.4f}")

    # Feature importance (based on the headcount prediction model)
    # Note: MultiOutputRegressor fits one model per target. We inspect one of them.
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': multi_output_ensemble.estimators_[0].named_estimators_['lgb'].feature_importances_
    }).sort_values('importance', ascending=False)

    print(f"\n=== Top 10 Important Features (for Headcount) ===")
    print(feature_importance.head(10))

    return multi_output_ensemble, feature_importance

# === Main Execution ===
print("=== Enhanced Ganpati Festival Crowd & Turnover Prediction ===")

# Prepare data
X_train, X_val, X_test, y_train, y_val, y_test, scaler = prepare_data_for_training(
    train_data, validation_data, test_data
)

# Train model
model, importance = train_enhanced_model(X_train, X_val, X_test, y_train, y_val, y_test)

print(f"\n=== Enhancement Complete ===")
print("Your model now predicts both headcount and next-hour turnover!")
print("Key improvements:")
print("- Advanced lag and rolling features")
print("- Cyclical time encoding")
print("- Weather impact scoring & interaction features")
print("- Multi-output ensemble of 3 models")
print("- Robust scaling")

=== Enhanced Ganpati Festival Crowd & Turnover Prediction ===
Creating enhanced features for headcount and turnover prediction...
Training features shape: (13855, 34)
Number of features: 34
Training enhanced multi-output ensemble model...

=== Validation Results ===
--- Headcount Prediction ---
MAE: 8,855.95 (Previous: 8,793)
RMSE: 20,410.04
R²: 0.8919

--- Turnover Prediction ---
MAE: 16,665.18
RMSE: 34,106.25
R²: 0.4805

=== Test Results ===
--- Headcount Prediction ---
MAE: 11,115.41
RMSE: 30,521.13
R²: 0.8675

--- Turnover Prediction ---
MAE: 21,340.27
RMSE: 49,614.17
R²: 0.4075

=== Top 10 Important Features (for Headcount) ===
                       feature  importance
23           headcount_lag_24h        2379
18            headcount_lag_1h        2317
24           headcount_lag_48h        2187
21            headcount_lag_6h        1864
19            headcount_lag_2h        1783
22           headcount_lag_12h        1682
20            headcount_lag_3h        1629
31  headcount_r